In [ ]:
#libraries 
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
import time
from webdriver_manager.chrome import ChromeDriverManager
from fake_useragent import UserAgent
import random
import requests
from bs4 import BeautifulSoup
import csv
import json

In [ ]:
headers= {
    "accept": "",
    "user-agent": ""
}


def get_source_html(url):
    
    #options
    options = webdriver.ChromeOptions()

    #user-agent
    options.add_argument("user-agent=")

    #disable webdriver
    options.add_argument("--disable-blink-features=AutomationControlled")

    #chromedriver
    driver = webdriver.Chrome(
        executable_path=r"",
        options=options
    )

    #selenium gains access to the site 
    try:
        driver.get(url=url)
        time.sleep(5)
    

        email_input = driver.find_element_by_name("email")
        email_input.clear()
        email_input.send_keys("")
        time.sleep(10)
    
        password_input = driver.find_element_by_name("password")
        password_input.clear()
        password_input.send_keys("")
        time.sleep(10)
        password_input.send_keys(Keys.ENTER)
    
        time.sleep(10)
    
        join_link = driver.find_element_by_link_text("Find Syndicates to Join").click()
        time.sleep(7)
        
        first_client = driver.find_element_by_xpath("/html/body/div[1]/div/main/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[2]/div/div/div/div[2]/div[2]/div/div[1]").click()
        time.sleep(5)
        personal_page = driver.find_element_by_xpath("/html/body/div[4]/div/div/div/div[2]/div/div[1]/div/div/div/div/div[2]/div/div[1]/a").click()
        time.sleep(7)
        name = driver.find_element_by_xpath("/html/body/div[1]/div[4]/div/div/div[2]/div[1]/div/div[2]/div/div/div[2]/div").get_attribute("class")
        time.sleep(7)
        print(name.text)
        
        #trying to find the info only with selenium
        
        #back_link = driver.execute_script("window.history.go(-1)")
        #time.sleep(7)
        #for i in range (10):
            #load_more_link = driver.find_element_by_xpath("//html/body/div[1]/div/main/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[2]/div/button").click()
            #time.sleep(6)
        #up_link = driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)  
        
        urls = []
        for i in range(200):
            first_client = driver.find_element_by_xpath(f'/html/body/div[1]/div/main/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[2]/div/div/div/div[2]/div[{i+1}]/div/div[1]/div').click()
            time.sleep(5)
            links = driver.find_element_by_xpath("/html/body/div[4]/div/div/div/div[2]/div/div[1]/div/div/div/div/div[2]/div/div[1]/a").get_attribute("href") 
            time.sleep(3)
            back = driver.find_element_by_xpath("/html/body/div[4]/div/div/div/div[1]/div").click()
            time.sleep(5) 
            urls.append(links)
            print(links)
        with open("three.txt", "w") as file:
            for url in urls:
                file.write(f"{url}\n")
    
    except Exception as ex:
        print(ex)
    finally:
        driver.close()
        driver.quit()
        

def main():
    get_source_html(url="https://angel.co/v/login")

if __name__ == "__main__":
    main()
    

### Collecting the information about the syndicates 

In [ ]:
headers = {
  'Connection': 'keep-alive',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.152 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
  'Sec-GPC': '1',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Referer': 'https://www.angellist.com',
  'Accept-Language': 'en-US,en;q=0.9,ru;q=0.8',
}


def get_data(file_path):
    with open(file_path) as file:
        urls = file.readlines()
        clear_urls_list = []
        for url in urls:
            url = url.strip()
            clear_urls_list.append(url)
    result_list = [] 
    urls_count = len(clear_urls_list)
    count = 1
        #print(clear_urls_list)
        #urls = [url.strip() for url in file.readlines()]
        #print(urls)

    for url in clear_urls_list:
        t = time.time() # get the current time
        resp = requests.get(url = url, headers=headers)
        response_time = time.time() - t
        time.sleep(round(response_time * 3)) # make a delay (round it up to a whole number)
        
        soup = BeautifulSoup(resp.content, 'html.parser')
        
        #syndicate name
        item_investors = soup.find_all('div', {'class': 'u-fontSize48 u-fontSize30SmOnly'})
        for tag1 in item_investors:
            investors = tag1.get_text().strip()
            print(investors)
        
        #typical investment
        item_typical_investments = soup.find_all('div', {'class': 'u-fontSize24'})[0]
        for tag2 in item_typical_investments:
            investments = tag2.get_text().strip()
            print(investments)
        
        #last deals
        item_last_deals = soup.find_all('div', {'class': 'u-fontSize24'})[1]
        for tag3 in item_last_deals:
            deals = tag3.get_text().strip()
            print(deals)
            
        #lps
        item_lps = soup.find_all('div', {'class': 'u-fontSize24'})[2]
        for tag4 in item_lps:
            lps = tag4.get_text().strip()
            print(lps)
        
        #print(investors + '  ' + investments + '  ' +  deals + '  ' + lps)
        
        #save to json
        result_list.append(
            {
                "investors": investors,
                "investments": investments,
                "deals": deals,
                "lps": lps
            }
        )
        
        #avoid the block
        time.sleep(random.randrange(2, 5))
    
        if count%10 == 0:
            time.sleep(random.randrange(10, 15))
    
        print(f"[+] Processed: {count}/{urls_count}")
    
        count += 1
    
    with open("namefile.json", "w", encoding="utf-8") as file:
        json.dump(result_list, file, indent=4, ensure_ascii = False)
    
    return "[INFO] Data collected successfully"
        
    
def main():
    get_data(file_path=r"where the Windows/OS finds the links.txt")

if __name__ == "__main__":
    main()
    